In [ ]:
import healpy
from cora.util import hputil
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import ticker
from ch_util import ephemeris as ephem, andata, tools
from caput.time import unix_to_skyfield_time
import h5py
import time
from datetime import datetime
from glob import glob
from ch_pipeline.core import telescope
from scipy.optimize import leastsq

%load_ext autoreload
%autoreload 2

from continuum_beam import *

%matplotlib inline
plt.rcParams.update({'figure.figsize': (16, 12), 'font.size': 20})

# Validate fitting scheme on simulated visibilities

### Load simulated visibilities

In [ ]:
sim_file = "/home/tristpinsm/scratch-fast/continuum_beam_data/sstreamgroup_0.h5"

In [ ]:
sim_vis = h5py.File(sim_file, 'r')

In [ ]:
sim_vis.keys()

In [ ]:
freq = sim_vis['index_map']['freq']['centre']

In [ ]:
start_time = time.time()
sim_time = np.array([ ephem.transit_times(r, start_time) for r in sim_vis['index_map']['ra'] ])

In [ ]:
plt.plot(sim_time)

In [ ]:
inputs = tools.get_correlator_inputs(datetime.now(), correlator='pathfinder')

In [ ]:
pos = tools.get_feed_positions(inputs)
pol = tools.get_feed_polarisations(inputs)

In [ ]:
prod_excl = []

ns_baselines = (pos[sim_vis['index_map/prod'][:,0],1]
                 - pos[sim_vis['index_map/prod'][:,1],1])
ew_baselines = (pos[sim_vis['index_map/prod'][:,0],0]
                 - pos[sim_vis['index_map/prod'][:,1],0])

pol_pair = np.empty(sim_vis['index_map/prod'].shape[0], dtype=[('pol_a', '<U1'), ('pol_b', '<U1')])
pol_pair['pol_a'] = pol[sim_vis['index_map/prod'][:,0]]
pol_pair['pol_b'] = pol[sim_vis['index_map/prod'][:,1]]

# exclude bad channels
prod_excl += list(np.where(np.logical_not(np.isfinite(ew_baselines + ns_baselines)))[0])
# exclude intercyl
prod_excl += list(np.where(np.abs(ew_baselines) > 5.)[0])
# exclude autos
prod_excl += list(np.where(ns_baselines + ew_baselines == 0)[0])
# exclude all but SS pol
prod_excl += list(np.where(np.logical_not(np.logical_and(pol_pair['pol_a'] == "S",
                                                         pol_pair['pol_b'] == "S")))[0])

# get unique values
prod_excl = set(prod_excl)

prod_sel = np.array([ p for p in range(sim_vis['index_map/prod'].shape[0]) if not p in prod_excl ])

### Compare to input beam

In [ ]:
path = telescope.CHIME.from_layout(datetime.now(), correlator='pathfinder')

In [ ]:
def model_beam(za, fwhm_fudge=0.7):
    fwhm = 2.0 * np.pi / 3.0 * fwhm_fudge
    
    alpha = np.log(2.0) / (2*np.tan(fwhm / 2.0)**2)

    return np.exp(-alpha*np.tan(za)**2)

In [ ]:
f_ind = 58  # 408MHz
vis = sim_vis['vis'][f_ind, prod_sel, :]
ns_baselines = ns_baselines[prod_sel]

In [ ]:
sim_model = ModelVis(freq=freq[f_ind])

In [ ]:
sim_model.set_baselines(ns_baselines)

In [ ]:
pos[sim_vis['index_map/prod'][prod_sel[0]]]

In [ ]:
for p in range(len(prod_sel)):
    plt.plot(np.abs(vis[p,:]))

In [ ]:
np.degrees(2 * np.sqrt(2*np.log(2)) * sim_model._res())

In [ ]:
max_za = 80.
# approx resolution for smoothed Haslam
fwhm_smoothing = np.degrees(2 * np.sqrt(2*np.log(2)) * sim_model._res())
num_pix = 2 * int(2 * max_za / fwhm_smoothing)
za = np.radians(np.linspace(-max_za, max_za, num_pix))
#time_slice = slice(test_ind-5, test_ind+5)
time_slice = slice(0,700)

In [ ]:
beam_sol = sim_model.fit_beam(sim_time[time_slice], vis[:,time_slice],
                               np.ones_like(vis[:,time_slice]),
                               num_pix, max_za=max_za, rcond=1e-6)

In [ ]:
plt.plot(za, beam_sol / beam_sol.max())
plt.plot(za, np.cos(za)*model_beam(za, fwhm_fudge=0.7))

In [ ]:
U, S, V = np.linalg.svd(sim_model.M)

In [ ]:
plt.plot(np.log10(S / S[0]))

In [ ]:
plt.imshow(np.log10(np.abs(np.dot(np.linalg.pinv(sim_model.M), sim_model.M))),
          extent=(-max_za,max_za,-max_za,max_za))
plt.colorbar()

#### Fit for FWHM

In [ ]:
fwhm_fit = leastsq(lambda t: beam_sol / beam_sol.max() - model_beam(za, t), 1.)

In [ ]:
fwhm_fit[0][0]

In [ ]:
plt.plot(za, beam_sol / beam_sol.max())
plt.plot(za, model_beam(za, fwhm_fit[0][0]))

#### Look at map slices

In [ ]:
sim_model._gen_basis(sim_time[time_slice], vis[:,time_slice],
                     num_pix, max_za=max_za)
model_basis = sim_model._basis.copy()
model_vis = np.sum(model_basis * model_beam(za), axis=2)

In [ ]:
#model_vis = sim_model.get_vis(sim_time[time_slice], vis[:,time_slice],
#                              num_pix, max_za=max_za)
model_map = np.dot(model_vis[:,0], np.exp(-2j * np.pi * ns_baselines[:,np.newaxis]
                                          / sim_model.wl * np.sin(za)[np.newaxis,:]))
vis_map = np.dot(vis[:,test_ind], np.exp(-2j * np.pi * ns_baselines[:,np.newaxis]
                                    / sim_model.wl * np.sin(za)[np.newaxis,:]))

In [ ]:
plt.subplot(2,1,1)
plt.plot(za/np.pi, model_map.real, label="Haslam")
plt.gca().yaxis.set_ticklabels([])
plt.legend()

plt.subplot(2,1,2)
plt.plot(za/np.pi, vis_map.real / vis_map.real.max(), label="sim")
plt.plot(za/np.pi, model_beam(za))
plt.gca().yaxis.set_ticklabels([])
plt.legend()

plt.xlabel(r"$\theta_k/\pi$")
#yfmt = ticker.ScalarFormatter()
#yfmt.set_powerlimits((-2,2))
#plt.gca().yaxis.set_major_formatter(yfmt)